In [ ]:

from typing import NamedTuple
def myOp(intArgs: int = 0, strArg: str = 'default-str') -> NamedTuple('outputs', [('Validation-Accuracy', float), ('Validation-Loss', float)]):

    home = '/home/jovyan'
    ## begin your code here
    ## ....
    ## end your code here
    
    
    ## use monitorCallback to let tintin to track your model performance, hence a comparison view can be used
    #class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    #        def on_epoch_end(self, epoch, logs=None):
    #            print("Training-Accuracy={:7.6f}".format(logs["accuracy"]))
    #            print("Training-Loss={:7.6f}".format(logs["loss"]))
    #            print("Validation-Accuracy={:7.6f}".format(logs["val_accuracy"]))
    #            print("Validation-Loss={:7.6f}".format(logs["val_loss"]))
    #callbacks = [..., LossAndErrorPrintingCallback(), ...]
    #    # Run training, with or without data augmentation.
    #    if not data_augmentation:
    #        print('Not using data augmentation.')
    #        model.fit(x_train, y_train,
    #                  batch_size=batch_size,
    #                  epochs=epochs,
    #                  validation_data=(x_test, y_test),
    #                  shuffle=True,
    #                  callbacks=callbacks)
    
    
    from collections import namedtuple
    outputs = namedtuple('outputs', ['Validation-Accuracy', 'Validation-Loss'])
    return outputs(output_model_dir, validAcc, validLoss)

In [ ]:
## to access assets files, use tintin-sdk (requires python >=3.7)
## see tintin-sdk: https://github.com/footprintAI/tintin-sdk
#!pip install tintin-sdk --user

In [ ]:
## write all your requirements.txt here if possible
with open("requirements.txt", "w") as f:
    f.write("kfp==0.5.1\n")
    f.write("h5py<3.0.0\n")
    f.write("keras==2.3.1\n")
    f.write("tintin-sdk\n")

!pip install -r requirements.txt --user --upgrade

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp
import kfp.compiler as compiler

In [ ]:
import os
pvcname = os.environ.get('TINTIN_SESSION_TEMPLATE_PVC_NAME')
generated_pipeline_zip_filename = os.environ.get('TINTIN_SESSION_TEMPLATE_GENERATED_PIPELINE_ZIP_FILENAME')
gpu_type_list_text = os.environ.get('TINTIN_SESSION_TEMPLATE_GPU_TYPE_LIST')
default_image = os.environ.get('TINTIN_SESSION_TEMPLATE_DEFAULT_IMAGE', 'footprintai/nvidia-tensorflow:19.12-tf1-py3')
mountPath = os.environ.get('TINTIN_SESSION_TEMPLATE_MOUNT_PATH', '/home/jovyan')



In [ ]:
## this section attach your operation function
## with kubeflow pipeline.

myComp = comp.func_to_container_op(myOp, 
                                      base_image=default_image,
                                      packages_to_install=["keras==2.3.1", "h5py<3.0.0"])

import kfp.dsl as dsl
@dsl.pipeline(
   name='Projectname pipeline',
   description='simple pipeline.'
)
def templated_pipeline_func(
    intArg=50,
    strArg='str',
):
    
    pvolumn_dict = {}
    pvolumn_dict[mountPath] = dsl.PipelineVolume(pvc=pvcname);
    
    ## do NOT edit this session unless you know what are you doing.
    train_task = myComp(intArgs, strArgs).add_pvolumes(pvolumn_dict)
    train_task = train_task.add_resource_request('cpu', '1')
    train_task = train_task.add_resource_limit('cpu', '1')
    train_task = train_task.add_resource_request('memory', '4Gi')
    train_task = train_task.add_resource_limit('memory', '4Gi')
    if len(gpu_type_list_text) > 0:
        gpu_type_list = gpu_type_list_text.split(',')
        # Set gpu type here. Default the first gpu type in user resource quota.
        # User could print out the gpu type list and pick up one to replace the following line.
        gpu_type = gpu_type_list[0]
        train_task = train_task.add_resource_request(gpu_type, '1')
        train_task = train_task.add_resource_limit(gpu_type, '1')
        # GPU company is from the GPU type. Used by the gpu driver
        gpu_company = gpu_type.split('/')[0] + '/gpu'
        train_task = train_task.add_resource_request(gpu_company, '1')
        train_task = train_task.add_resource_limit(gpu_company, '1')
    model_name = os.environ.get('TINTIN_SESSION_TEMPLATE_MODEL_NAME', 'resnet_graphdef')
    train_task = train_task.add_pod_annotation('tintin.footprint-ai.com/session-model-name', model_name)
compiler.Compiler().compile(templated_pipeline_func, generated_pipeline_zip_filename)